In [32]:
import numpy as np
import pickle
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils

## Data

In [2]:
fn = 'data/shakespeare.txt'
sentences = []
all_chars = []
with open(fn, 'r') as f:
    for line in f:
        if len(line.strip()) > 3:
            if line[:2] == '  ': line = line[2:]
            sentences.append(line.lower())
            all_chars += list(line.lower())

In [3]:
# Create vocabulary.
chars = sorted(list(set(all_chars)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(all_chars)
n_vocab = len(chars)
print('Total characters: %d' % n_chars)
print('Total vocab: %d' % n_vocab)

Total characters: 93673
Total vocab: 38


In [4]:
# Construct dataset.
seq_len = 40
step_size = 2
dataX = []
dataY = []
for i in range(0, n_chars - seq_len, step_size):
    seq_in = all_chars[i: i + seq_len]
    seq_out = all_chars[i + seq_len]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print('Total patterns: %d' % n_patterns)

Total patterns: 46817


In [5]:
X = np.reshape(dataX, (n_patterns, seq_len, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

## Train model

In [14]:
dim = 200
model = Sequential()
model.add(LSTM(dim, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 200)               161600    
_________________________________________________________________
dense_2 (Dense)              (None, 38)                7638      
Total params: 169,238
Trainable params: 169,238
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
fn = 'lstm_weights/best_model_stepsize2_dim%d.h5' % dim
mc = ModelCheckpoint(fn, monitor='acc', save_best_only=True)
es = EarlyStopping(monitor='acc', baseline=0.6, patience=0)
callbacks_list = [mc]

In [16]:
model.fit(X, y, epochs=200, batch_size=256, callbacks=callbacks_list)

Epoch 1/200
46817/46817 [==============================] - 16s 350us/step - loss: 2.9810 - acc: 0.1685
Epoch 2/200
46817/46817 [==============================] - 16s 345us/step - loss: 2.7869 - acc: 0.2234
Epoch 3/200
46817/46817 [==============================] - 16s 347us/step - loss: 2.7072 - acc: 0.2401
Epoch 4/200
46817/46817 [==============================] - 17s 364us/step - loss: 2.6545 - acc: 0.2510
Epoch 5/200
46817/46817 [==============================] - 17s 357us/step - loss: 2.6101 - acc: 0.2625
Epoch 6/200
46817/46817 [==============================] - 16s 352us/step - loss: 2.5644 - acc: 0.2683
Epoch 7/200
46817/46817 [==============================] - 16s 347us/step - loss: 2.5293 - acc: 0.2768
Epoch 8/200
46817/46817 [==============================] - 16s 350us/step - loss: 2.4975 - acc: 0.2843
Epoch 9/200
46817/46817 [==============================] - 17s 353us/step - loss: 2.4651 - acc: 0.2875
Epoch 10/200
46817/46817 [==============================] - 17s 353us/ste

46817/46817 [==============================] - 17s 365us/step - loss: 1.4152 - acc: 0.5641
Epoch 80/200
46817/46817 [==============================] - 17s 369us/step - loss: 1.3910 - acc: 0.5712
Epoch 81/200
46817/46817 [==============================] - 18s 383us/step - loss: 1.4058 - acc: 0.5672
Epoch 82/200
46817/46817 [==============================] - 17s 367us/step - loss: 1.4160 - acc: 0.5635
Epoch 83/200
46817/46817 [==============================] - 18s 374us/step - loss: 1.4523 - acc: 0.5515
Epoch 84/200
46817/46817 [==============================] - 17s 366us/step - loss: 1.3877 - acc: 0.5722
Epoch 85/200
46817/46817 [==============================] - 17s 369us/step - loss: 1.3854 - acc: 0.5725
Epoch 86/200
46817/46817 [==============================] - 17s 365us/step - loss: 1.3651 - acc: 0.5808
Epoch 87/200
46817/46817 [==============================] - 17s 361us/step - loss: 1.3501 - acc: 0.5829
Epoch 88/200
46817/46817 [==============================] - 17s 372us/step - 

46817/46817 [==============================] - 18s 379us/step - loss: 1.1444 - acc: 0.6404
Epoch 158/200
46817/46817 [==============================] - 18s 384us/step - loss: 1.1226 - acc: 0.6487
Epoch 159/200
46817/46817 [==============================] - 18s 379us/step - loss: 1.1697 - acc: 0.6326
Epoch 160/200
46817/46817 [==============================] - 18s 392us/step - loss: 1.2593 - acc: 0.6060
Epoch 161/200
46817/46817 [==============================] - 18s 378us/step - loss: 1.1644 - acc: 0.6326
Epoch 162/200
46817/46817 [==============================] - 18s 377us/step - loss: 1.1454 - acc: 0.6410
Epoch 163/200
46817/46817 [==============================] - 18s 380us/step - loss: 1.1445 - acc: 0.6393
Epoch 164/200
46817/46817 [==============================] - 18s 377us/step - loss: 1.1545 - acc: 0.6387
Epoch 165/200
46817/46817 [==============================] - 18s 376us/step - loss: 1.1322 - acc: 0.6399
Epoch 166/200
46817/46817 [==============================] - 18s 378u

## Generate text

In [39]:
def sample(preds, temperature=1.0):
    # Helper function to sample from a probability array
    # Source: https://keras.io/examples/lstm_text_generation/
    preds = np.exp(preds / temperature)
    probs = preds / np.sum(np.exp(preds / temperature))
    return np.argmax(probs)

In [19]:
dim = 200
ckpt = "lstm_weights/best_model_stepsize2_dim%d.h5" % dim
model.load_weights(ckpt)
model.evaluate(X, y)

46817/46817 [==============================] - 12s 246us/step


[1.0193825700311212, 0.6822949051856995]

In [54]:
seed = "shall i compare thee to a summer's day?\n"
pattern = [char_to_int[char] for char in seed]
output_seq = pattern

In [55]:
tmp = 1.5
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)[0]
    index = sample(prediction, temperature=tmp)
    #index = np.argmax(prediction)
    pattern.append(index)
    pattern = pattern[1:]
    output_seq.append(index)

In [56]:
output = [int_to_char[i] for i in output_seq]
output = ''.join(output)
print(output)

shall i compare thee to a summer's day?
tthou hrsw pong,
and a map d nonw that woue dx the gail,
if thou ae wtatthngd mine sha lise eaik uhich:
bu aly she mot my lifd of my love dossont dnrh.
that you tealo the farhe with hare iu blate,
andcl io ios shat which fote lo not sooe?
mo eosh midht that thay fole cace mo jove,
shat you were uourhy sooe vorthp ther des,
whth niny the whrd poopui pow my lofw.
hoo thened iof nfres to the meny sr me.
ty ferd io my aeduty lole blltert mf,
howes,heented wo tey,st ieve wet the foowi,
and the wrrld wou dearying oo the werd roeie
to mesorreng afaio hv seyse shat tie soaes,
gor the vorld soon gym to she menhlsg nrn,
yhth all the mars,
and a gaievu sie oor ro shmngsed to woy.
and eea
oyt opw by the vhil,
wher i bm cosh iy she menhsing thee,
and iaale yotr cruk ountese pf thi thahnu
than thay forwu, and av a sart,
the eoos hnv jovesthng goaee,
oor rrou lookeng worthy moow brofkrs fedr,
and the whrd pfrppnee eyes hot shy wonklf:
no eesties thene iarh iiv 